# Tools and Routing

In [113]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [114]:
from langchain.agents import tool

@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [115]:
search.name

'search'

In [116]:
search.description

'search(query: str) -> str - Search for weather online'

In [117]:
import json

# Note: in json.dumps the base indent is 0 which does not look good
print(json.dumps(search.args, indent=2))

{
  "query": {
    "title": "Query",
    "type": "string"
  }
}


In [118]:
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [119]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [120]:
print(json.dumps(search.args, indent=2))

{
  "query": {
    "title": "Query",
    "description": "Thing to search for",
    "type": "string"
  }
}


In [121]:
search.invoke("sf")

'42f'

In [122]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # Note: you don't need to understand this logic just understand the above langchain tool part
    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [123]:
get_current_temperature.name

'get_current_temperature'

In [124]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [125]:
print(json.dumps(get_current_temperature.args, indent=2))

{
  "latitude": {
    "title": "Latitude",
    "description": "Latitude of the location to fetch weather data for",
    "type": "number"
  },
  "longitude": {
    "title": "Longitude",
    "description": "Longitude of the location to fetch weather data for",
    "type": "number"
  }
}


In [126]:
from langchain_core.utils.function_calling import convert_to_openai_function

print(json.dumps(convert_to_openai_function(get_current_temperature), indent=2))

{
  "name": "get_current_temperature",
  "description": "get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.",
  "parameters": {
    "type": "object",
    "properties": {
      "latitude": {
        "description": "Latitude of the location to fetch weather data for",
        "type": "number"
      },
      "longitude": {
        "description": "Longitude of the location to fetch weather data for",
        "type": "number"
      }
    },
    "required": [
      "latitude",
      "longitude"
    ]
  }
}


In [127]:
# These are the coordinates for Mumbai
get_current_temperature.invoke({"latitude": 19.0760, "longitude": 72.8777})
# Note: You are allowed to do this because of the @tool decorator

'The current temperature is 32.4°C'

In [128]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [129]:
search_wikipedia.name

'search_wikipedia'

In [130]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [131]:
print(json.dumps(convert_to_openai_function(search_wikipedia), indent=2))

{
  "name": "search_wikipedia",
  "description": "search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string"
      }
    },
    "required": [
      "query"
    ]
  }
}


In [132]:
search_wikipedia.invoke({"query": "langchain"})
# Note: You are allowed to do this because of the @tool decorator

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers which e

In [133]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [134]:
# Note: This is similar to the swagger, the documentation for your backend
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [135]:
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [136]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [137]:
print(json.dumps(pet_openai_functions, indent=2))

[
  {
    "name": "listPets",
    "description": "List all pets",
    "parameters": {
      "type": "object",
      "properties": {
        "params": {
          "type": "object",
          "properties": {
            "limit": {
              "type": "integer",
              "maximum": 100.0,
              "schema_format": "int32",
              "description": "How many items to return at one time (max 100)"
            }
          },
          "required": []
        }
      }
    }
  },
  {
    "name": "createPets",
    "description": "Create a pet",
    "parameters": {
      "type": "object",
      "properties": {}
    }
  },
  {
    "name": "showPetById",
    "description": "Info for a specific pet",
    "parameters": {
      "type": "object",
      "properties": {
        "path_params": {
          "type": "object",
          "properties": {
            "petId": {
              "type": "string",
              "description": "The id of the pet to retrieve"
            }
          },

In [138]:
from langchain_openai import ChatOpenAI

In [139]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [140]:
model.invoke("what are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 123, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-121c38c5-d3cc-4a91-a717-b0c6d2ce90f3-0')

In [141]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 126, 'total_tokens': 145}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-a1ec1526-7a26-4c74-8d6e-2099db1ecc7f-0')

### Routing

This is an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [142]:
functions = [
    convert_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [143]:
model.invoke("what is the weather in sf right now")
# Note: This will find co-ordinates for san francisco and create parameters for get_current_temperature

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 129, 'total_tokens': 154}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-87e11339-5b38-4894-91a0-207ed6798aef-0')

In [144]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 125, 'total_tokens': 141}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d0c87d6d-05b9-4721-9208-2101d29e4f9e-0')

In [145]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [146]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 137, 'total_tokens': 162}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-bf26e0a6-c873-4ab5-8d6a-8540641d2cd6-0')

In [147]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [148]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [149]:
type(result)
# There was a function call for this result hence the type AgentAction 

langchain_core.agents.AgentActionMessageLog

In [150]:
result.tool
# this is the selected function in form of a string

'get_current_temperature'

In [151]:
result.tool_input
# you need to invoke the above selected function with this input this is in form of a dict not a string

{'latitude': 37.7749, 'longitude': -122.4194}

In [152]:
get_current_temperature.invoke(result.tool_input)

'The current temperature is 12.9°C'

In [153]:
result = chain.invoke({"input": "hi!"})

In [154]:
type(result)
# There was no function call for this result hence the type AgentFinish 

langchain_core.agents.AgentFinish

In [155]:
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

In [156]:
from langchain_core.agents import AgentFinish

# This is where the actual result is obtained
# This route function actually acts on the result of the language model and does the corresponding steps
def route(result):
    if isinstance(result, AgentFinish):
        # This means that no functions were called and the output content is present and not an empty string
        return result.return_values['output']
    else:
        # Note this is needed since result.tool is a string 
        # and we are mapping the string to the correct function
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        # Note this is the same as we have done previously
        return tools[result.tool].invoke(result.tool_input)

In [157]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [158]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})
result

'The current temperature is 12.9°C'

In [159]:
result = chain.invoke({"input": "What is langchain?"})
result

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\n\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers which e

In [160]:
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'